In [1]:
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import expr
from pyspark.sql.functions import *
import pandas as pd
import time
sc.stop()
sc =SparkContext()
sqlCtx = SQLContext(sc)


In [4]:
rdd = sc.textFile("gs://ds-url-catag/adid_ppid_segment_check/request_2_mark/*.csv") \
    .map(lambda line: line.split(","))


In [5]:
first_line = rdd.first()

In [6]:
ppids = rdd.filter(lambda line: line != first_line).toDF(['adid'])

In [7]:
ppids.count()

2806982

In [8]:
ppids.dropDuplicates().count()

585918

In [9]:
final_ppids = ppids.where(col('adid') != '""')

In [10]:
final_ppids.count()

2806982

In [11]:
#convert to ppids
from hashlib import sha1
import binascii
import re 
SALT_A = "77EfzyR%iZtyTFAK7lyZ_X=B7e^b1atUx%W+Rx0A3ja~b5yjGL-q==po%nkL"

def adid_to_ppid(adid):
    sha1hash = []
    pattern = re.compile("^[a-zA-Z0-9\-]{8}-[a-zA-z0-9]{4}-[a-zA-z0-9]{4}-[a-zA-z0-9]{4}-[a-zA-z0-9]{12}")
    if re.match(pattern,adid) is not None:
        text = adid + SALT_A
        sha1_dig = sha1(text).digest()
        sha1hash = [ord(c) for c in sha1_dig]
    return binascii.hexlify(bytearray(sha1hash))

adid_ppid_udf = udf(adid_to_ppid, StringType())

In [12]:
final_ppids= final_ppids.withColumn('ppid',adid_ppid_udf(col('adid')))

In [14]:
final_ppids.show()

+--------------------+--------------------+
|                adid|                ppid|
+--------------------+--------------------+
|0f677cad-1e34-45c...|3904def9d176c888b...|
|01177995-6580-458...|ecfcf2c208203d743...|
|2c07d77b-3902-4c6...|8e1b44be7c5026621...|
|8ca0d23b-4d31-407...|6fc2103ce9e1af2d4...|
|1e9cc12e-2ce9-4f4...|287cb5472caa8ab5d...|
|f4231dc6-fd64-40c...|e0b807d7f5225a452...|
|c33fa70d-b6e4-42a...|c8dd2a55e8b518169...|
|aef7a87b-1400-45b...|9d06e9c5177369464...|
|9ad11051-e36d-42a...|3a4d890b21dabd827...|
|eb67064a-9afe-47d...|e5122bc04c39eb600...|
|95f76851-85ce-484...|ee45c165c794907a6...|
|d1675846-e5c3-4a6...|5a36100c7334137a3...|
|5bcdbebb-9481-404...|5cbda3e1bdabeddd9...|
|5a59927d-49d8-4b4...|775d27e9e70fdfb3a...|
|b7109c9f-e017-474...|c5f508f06ca0c322e...|
|38ee8250-b7ab-48a...|0abe364ad934fc243...|
|a4b65068-d9c4-41b...|cc11b1bfff2f1ba3e...|
|b33ea461-1fd0-418...|b1e845c33e9506fba...|
|6efddc14-3eef-467...|4429a1c2c10a6d1c6...|
|24a710de-c0c3-408...|77644eee20

In [13]:
final_ppids.select('ppid').dropDuplicates().count()

585910

In [15]:
final_ppids.select('ppid').dropDuplicates().coalesce(10).write.mode('overwrite').text("gs://ds-url-catag/adid_ppid_segment_check/request_2_mark/results/")